In [33]:
import time
import numpy as np
import pandas as pd
import turicreate as tc
import mysql.connector as db
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [34]:
mydb = db.connect(
  host = "localhost",
  user = "root",
  passwd = "",
  database = "ecomm_ml"
)

lineItems = mydb.cursor()
lineItems.execute("SELECT `customer_id`, `product_id`, `quantity` FROM line_items")
lineItems = lineItems.fetchall()

customers = mydb.cursor()
customers.execute("SELECT `id` from `customers`")
customers = customers.fetchall()

In [35]:
lineItems = pd.DataFrame(lineItems,
                         columns=['customer_id', 'product_id', 'quantity'])

In [36]:
customers_list = []
for i in customers:
    customers_list.append(i[0])

In [37]:
# print(lineItems.shape)
# lineItems.head(15)

In [95]:
lineItems = lineItems.groupby(['customer_id', 'product_id'],
                             as_index=False)['quantity'].sum()
lineItems.head(15)

,customer_id,product_id,quantity
0,1,1466095829091,1
1,1,1466095960163,1
2,1,1466096189539,1
3,2,1466095894627,1
4,2,1466095960163,1
5,3,1466095829091,2
6,4,1466095861859,3
7,4,1466095894627,1
8,4,1466095927395,2
9,4,1466096418915,1


In [96]:
matrix = pd.pivot_table(lineItems, values='quantity', index='customer_id', columns='product_id')
# print(matrix.shape)
matrix.head()

product_id,11525025414,1239867981830,1466095829091,1466095861859,1466095894627,1466095927395,1466095960163,1466096189539,1466096255075,1466096287843,...,1466318520419,15486872454,15571182086,15571187078,33141363974,33141385926,844580651014,9672299203,9672385859,9672405955
customer_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3.0,1.0,2.0,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
# https://stats.stackexchange.com/questions/70801/how-to-normalize-data-to-0-1-range
matrix_normalised = (matrix-matrix.min())/(matrix.max()-matrix.min())
print(matrix_normalised.shape)
matrix_normalised.head()

(8135, 25)


product_id,11525025414,1239867981830,1466095829091,1466095861859,1466095894627,1466095927395,1466095960163,1466096189539,1466096255075,1466096287843,...,1466318520419,15486872454,15571182086,15571187078,33141363974,33141385926,844580651014,9672299203,9672385859,9672405955
customer_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0.142857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,0.4,0.0,0.066667,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
new_matrix = matrix_normalised.reset_index()
new_matrix.index.names = ['minmax_quantity']
new_matrix_two = pd.melt(new_matrix, id_vars=['customer_id'], value_name='minmax_quantity').dropna()
new_matrix_two

,customer_id,product_id,minmax_quantity
40,45,11525025414,0.0
100,112,11525025414,0.0
101,113,11525025414,0.0
109,121,11525025414,0.0
232,266,11525025414,0.0
240,276,11525025414,0.0
251,288,11525025414,0.0
284,327,11525025414,0.0
313,362,11525025414,0.0
374,441,11525025414,0.0


In [41]:
# d = matrix_norm.reset_index()
# d.index.names = ['something_new']
# data_norm = pd.melt(d, id_vars=['customer_id'], value_name='something_new').dropna()
# print(data_norm.shape)
# data_norm.head(80)

In [42]:
train, test = train_test_split(lineItems, test_size = .2)
print(train.shape, test.shape)

(11297, 3) (2825, 3)


In [43]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [49]:
train_data

customer_id,product_id,quantity
603,1466096189539,2
1125,1466095829091,1
6905,1466096189539,2
393,1466096386147,1
8547,1466095927395,1
6317,1466096189539,2
2230,1466096189539,1
5774,1466095894627,2
7840,1466096189539,1
3799,15571182086,1


In [50]:
test_data

customer_id,product_id,quantity
4713,1466096189539,1
4584,1466095861859,2
1041,1466095829091,1
7163,1466095861859,1
6208,1466095960163,1
5438,1466095960163,1
9005,1466096189539,1
7474,1466096189539,1
8912,1466096189539,2
7885,1466095960163,1


In [46]:
popularity_model = tc.popularity_recommender.create(train_data,
                                                    user_id="customer_id",
                                                    item_id="product_id")

Warning: Column 'quantity' ignored.

To use this column as the target, set target = "quantity" and use a method that allows the use of a target.

Preparing data set.

Data has 11297 observations with 7305 users and 25 items.

Data prepared in: 0.031936s

11297 observations to process; with 25 unique items.

In [47]:
popularity_recommendations = popularity_model.recommend(users=customers_list,
                                                       k=10)

recommendations finished on 1000/9265 queries. users per second: 215517

recommendations finished on 2000/9265 queries. users per second: 332005

recommendations finished on 3000/9265 queries. users per second: 287384

recommendations finished on 4000/9265 queries. users per second: 330825

recommendations finished on 5000/9265 queries. users per second: 374756

recommendations finished on 6000/9265 queries. users per second: 408219

recommendations finished on 7000/9265 queries. users per second: 401653

recommendations finished on 8000/9265 queries. users per second: 425193

recommendations finished on 9000/9265 queries. users per second: 449147

In [48]:
popularity_recommendations.print_rows(50)

+-------------+---------------+--------+------+
| customer_id |   product_id  | score  | rank |
+-------------+---------------+--------+------+
|      1      | 1466095861859 | 927.0  |  1   |
|      1      | 1466095894627 | 756.0  |  2   |
|      1      | 1466096386147 | 489.0  |  3   |
|      1      | 1466095927395 | 473.0  |  4   |
|      1      | 1466096353379 | 445.0  |  5   |
|      1      | 1466096320611 | 222.0  |  6   |
|      1      | 1466096255075 | 191.0  |  7   |
|      1      | 1466096451683 | 171.0  |  8   |
|      1      | 1466318520419 | 147.0  |  9   |
|      1      | 1466096418915 | 132.0  |  10  |
|      2      | 1466096189539 | 3515.0 |  1   |
|      2      | 1466095960163 | 1609.0 |  2   |
|      2      | 1466095829091 | 1453.0 |  3   |
|      2      | 1466095861859 | 927.0  |  4   |
|      2      | 1466096386147 | 489.0  |  5   |
|      2      | 1466095927395 | 473.0  |  6   |
|      2      | 1466096353379 | 445.0  |  7   |
|      2      | 1466096320611 | 222.0  |